# Global variables

In [ ]:
from imports import *
plt.rc('figure', max_open_warning=200)
%matplotlib notebook
sns.set_theme()
import plotly.graph_objs as go
import plotly.express as px

In [ ]:
# Initialize path variables for main folders

pwd = os.getcwd()
    
# Path for loading epoch data and features
ft_dir_path = pwd + '/features'
print(ft_dir_path)


In [ ]:
# Frequency bands

bands = [(0.9, 4, 'Delta (0.9-4 Hz)', 'D'), (4, 8, 'Theta (4-8 Hz)', 'T'), (8, 14, 'Alpha (8-14 Hz)', 'A'), 
         (14, 25, 'Beta (14-25 Hz)', 'B'), (25, 40, 'Gamma (25-40 Hz)', 'G')]
str_freq = [bands[i][3] for i in range(len(bands))]
n_freq = len(str_freq)


# Loading data

In [ ]:
# Loading epochs (Meditator data)

ft_dir_path = '/home/lipperrdino/verenv/features/'
sec5_epochs = mne.read_epochs(os.path.join(ft_dir_path, 'epochs.fif'))
print(sec5_epochs.get_data().shape)

sampling_rate = sec5_epochs.info['sfreq']
n_samples = sec5_epochs.__len__()
n_times = len(sec5_epochs.get_data()[0,0,:])

ch_names = sec5_epochs.ch_names
n_channels = len(ch_names)

In [ ]:
# Loading main features DataFrames

df_ft_psd_all_db = pd.read_feather(os.path.join(ft_dir_path, 'df_ft_psd_all_db.feather'))
df_ft_coh_ind_all = pd.read_feather(os.path.join(ft_dir_path, 'df_ft_coh_ind_all.feather'))
df_ft_plv_ind_all = pd.read_feather(os.path.join(ft_dir_path, 'df_ft_plv_ind_all.feather'))

# Clusterization functions

In [ ]:
# Функция для расчёта оценок качетва кластеризации
def cluster_metrics_noground(name, data, labels_pred):
    results = [name]
    df_data = pd.DataFrame(columns=['Method', 'Silh', 'Cal-Har', 'Dav-Bold'])

    # Define the metrics which require only data and predicted labels
    cluster_metrics = [
        metrics.silhouette_score,
        metrics.calinski_harabasz_score,
        metrics.davies_bouldin_score
    ]

    results += [m(data, labels_pred) for m in cluster_metrics]
    df_data.loc[0] = results
    return df_data

In [ ]:
def apply_cluster_method(data, cl_method, name, df_metrics, **kwargs):
    method = cl_method(**kwargs).fit(data)
    df = cluster_metrics_noground(name, data, method.labels_)
    df_metrics = df_metrics.drop(df_metrics[df_metrics['Method']==name].index, errors='ignore')
    df_metrics = pd.concat([df_metrics, df], ignore_index=True)
    return method, df_metrics

In [ ]:
# Calculating clustering noground metrics for adjacent pairs of stages (Silh, Cal-Har, Dav-Bold)
def calc_stage_metr_noground(df_features, st_edges):
    df_metrics = pd.DataFrame(columns=['Method', 'Silh', 'Cal-Har', 'Dav-Bold'])
    for _st in range(1, len(st_edges)-1):
        name = 'St'+str(_st)+'_St'+str(_st+1)
        data = df_features.iloc[st_edges[_st-1]:st_edges[_st+1]].reset_index(drop=True)
        labels = [0 for i in range(st_edges[_st]-st_edges[_st-1])] + [1 for i in range(st_edges[_st+1]-st_edges[_st])]
        df = cluster_metrics_noground(name, data, labels)
        df_metrics = df_metrics.drop(df_metrics[df_metrics['Method']==name].index, errors='ignore')
        df_metrics = pd.concat([df_metrics, df], ignore_index=True)
    df_metrics.rename(columns={'Method': 'Stages'}, inplace=True)
    return df_metrics

## Downloading precalculated data

In [ ]:
df_st_edges_best = pd.read_csv('/home/lipperrdino/verenv/notebooks/df_st_edges_best.csv')
df_st_edges = pd.read_csv('/home/lipperrdino/verenv/notebooks/df_st_edges.csv')
df_features = pd.read_csv('/home/lipperrdino/verenv/notebooks/df_features.csv')
df_st_edge_metrics = pd.read_csv('/home/lipperrdino/verenv/notebooks/df_st_edge_metrics12.csv')

In [ ]:
df_st_edges_best['St_edges'] = df_st_edges_best['St_edges'].map(lambda x: list(map(int, x[1:-1].split(', '))))

In [ ]:
df_st_edges['St_edges'] = df_st_edges['St_edges'].map(lambda x: list(map(int, x[1:-1].split())))

In [ ]:
n_stages = 6 # expert choise of result number of stages

st_len = df_st_edges_best[df_st_edges_best['N_stages']==n_stages]['St_len_min'].iloc[0]
k_nb = df_st_edges_best[df_st_edges_best['N_stages']==n_stages]['K_nb_max'].iloc[0]
n_cl = df_st_edges_best[df_st_edges_best['N_stages']==n_stages]['N_cl_max'].iloc[0]

# Cluster center type
cl_center_type = df_st_edges_best[df_st_edges_best['N_stages']==n_stages]['Cl_cen'].iloc[0]
print(st_len, k_nb, n_cl, cl_center_type)

# Stage boundary epoch numbers
st_edges_result = df_st_edges_best[df_st_edges_best['N_stages']==n_stages]['St_edges'].iloc[0]

print(st_edges_result)

st_edges_all = []
st_edges_list = []
st_edges_list += (df_st_edges[(df_st_edges['Len_min']==st_len) & (df_st_edges['K_neighb']<=k_nb) &
                  (df_st_edges['N_clusters']<=n_cl)]['St_edges'].tolist())
for _st_edges in st_edges_list:
    st_edges_all += _st_edges[1:-1]
st_edges_all = sorted(st_edges_all)
df_st_edges_all = pd.DataFrame(st_edges_all)
#print(st_edges_all)



# Clustering stage edges
cl_name = 'kmeans_edges_'+str(n_stages-1)+'_'+str(st_len)+'_'+str(k_nb)
cluster_method, df_st_edge_metrics = cq.apply_cluster_method(data=df_st_edges_all, cl_method=KMeans, 
                                                            name=cl_name, df_metrics=df_st_edge_metrics, 
                                                            n_clusters=n_stages-1, random_state=0)
edg_labels_all = cluster_method.labels_
   
# Plotting st_edges_all
x = st_edges_all
y = [0]*len(x)
s = [1.5*x.count(x[i]) for i in range(len(x))]
fig, ax = plt.subplots(figsize=(6,4))
ax.scatter(x,y, s=s)
ax.set(xlabel='Epoches')
ax.tick_params(axis='both', labelsize=11, direction='in')
fig.suptitle('Stage edges merged for all parameters', fontsize=16)
# plt.savefig(subj_dir_name+' Stage edges merged.png')

# Plotting clusters    
x = st_edges_all
y = cluster_method.labels_
s = [1.5*x.count(x[i]) for i in range(len(x))]
fig, ax = plt.subplots(figsize=(6,4))
ax.scatter(x,y, s=s)
ax.set(xlabel='Epoches', ylabel='Clusters')
ax.tick_params(axis='both', labelsize=11, direction='in')
fig.suptitle('Stage edge clusters', fontsize=16)
# plt.savefig(subj_dir_name+' Stage edge clusters.png')


In [ ]:
# Find clustering statistic parameters (Main data)

df_st_edg_stats = pd.DataFrame(columns=['Cluster name', 'Median', 'Mean', 'Mode', 'Mode probability', 'Standard deviation', 
                                        'Median 0.95 confidence interval', 'Cluster size'])
st_clusters = []
print(n_stages)
for _st in range(n_stages-1):
    st_clusters.append([st_edges_all[i] for i in np.where(cluster_method.labels_ == _st)[0]])
    mode, count = sp.stats.mode(st_clusters[_st])
    conf_int_median = spr.median_confidence_interval(pd.Series(st_clusters[_st]), cutoff=0.95)
    print(conf_int_median)
    new_row = {'Cluster name': 'Boundary cluster '+str(_st), 'Median': int(np.median(st_clusters[_st])), 'Mean': int(np.array(st_clusters[_st]).mean()), 'Mode': mode[0], 
               'Mode probability': count[0]/len(st_clusters[_st]), 'Standard deviation': int(np.array(st_clusters[_st]).std()), 
               'Median 0.95 confidence interval': str(conf_int_median), 'Cluster size': len(st_clusters[_st])}

    df_st_edg_stats = pd.concat([df_st_edg_stats, pd.DataFrame(new_row, index=[0])], ignore_index = True)


df_st_edg_stats = df_st_edg_stats.sort_values('Median', ignore_index=True)


display(df_st_edg_stats)



In [ ]:
# Main data
df_adj_st_metr = calc_stage_metr_noground(df_features, st_edges_result)
st_metr_silh = df_adj_st_metr['Silh'].to_numpy()
st_metr_calh = df_adj_st_metr['Cal-Har'].to_numpy()
st_metr_davb = df_adj_st_metr['Dav-Bold'].to_numpy()

st_dist_ward, st_dist_centr = spr.calc_stage_distances(df_features, st_edges_result)
df_adj_st_metr['Centr'] = st_dist_centr
df_adj_st_metr['Ward'] = st_dist_ward

display(df_adj_st_metr)

In [ ]:
def feature_plot(df_ft, df_ft_clust, df_ft_clust_stats, st_edges_result, feature_name, axs, row):
    df_ft_clust = [df_ft.iloc[smin:(smax+1), :] for (smin, smax, _) in st_bands]
    df_ft_clust_stats = [df_ft_clust[_cl].describe() for _cl in range(n_stages)]
    for i in range(n_freq):
        y = df_ft.to_numpy()[:,i]  
        axs[row,i].plot(y, color='blue')
        axs[row,i].vlines(x=st_edges_result[1:-1], ymin=np.min(y), ymax=np.max(y), color='black', linewidth=1.8)
        axs[row,i].set_title(feature_name + bands[i][2])
        axs[row,i].tick_params(axis='both', labelsize=8, direction='in')

        # Error bars (statistic plots)
        x_st = [(smin + smax)/2 for (smin, smax, _) in st_bands]
        y_st = [df_ft_clust_stats[_cl].loc['mean'][i] for _cl in range(n_stages)]
        yerr = np.array([df_ft_clust_stats[_cl].loc[['25%','75%']].to_numpy() for _cl in range(n_stages)])[:,:,i].transpose(1,0)
        yerr[0,:] = y_st - yerr[0,:]
        yerr[1,:] = yerr[1,:] - y_st
        
        axs[row,i].errorbar(x_st, y_st, np.abs(yerr), capsize=4, linewidth=2.5, color='red')


In [ ]:
# Plotting overall PSD, Coherence and PLV with stage bounds

ind_rate = '07'

st_bands, st_labels = spr.form_stage_bands(st_edges_result, n_samples)

# Features DataFrames
df_ft_psd = df_ft_psd_all_db
df_cols = [col for col in df_ft_coh_ind_all.columns if (ind_rate in col)]
df_ft_coh = df_ft_coh_ind_all[df_cols]
df_cols = [col for col in df_ft_plv_ind_all.columns if (ind_rate in col)]
df_ft_plv = df_ft_plv_ind_all[df_cols]

# Initialize plots
n_cols = n_freq
n_rows = 3
fig, axs = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(3*n_cols, 3*n_rows))
plt.subplots_adjust(left=0.02)

# PSD plots (1st row)
df_ft_clust = [df_ft_psd.iloc[smin:(smax+1), :] for (smin, smax, _) in st_bands]
df_ft_clust_stats = [df_ft_clust[_cl].describe() for _cl in range(n_stages)]
feature_plot(df_ft_psd, df_ft_clust, df_ft_clust_stats, st_edges_result, 'PSD', axs, 0)

# Coherence plots (2nd row)
df_ft_clust = [df_ft_coh.iloc[smin:(smax+1), :] for (smin, smax, _) in st_bands]
df_ft_clust_stats = [df_ft_clust[_cl].describe() for _cl in range(n_stages)]
feature_plot(df_ft_coh, df_ft_clust, df_ft_clust_stats, st_edges_result, 'Coh_Ind_', axs, 1)

# PLV plots (3rd row)
df_ft_clust = [df_ft_plv.iloc[smin:(smax+1), :] for (smin, smax, _) in st_bands]
df_ft_clust_stats = [df_ft_clust[_cl].describe() for _cl in range(n_stages)]
feature_plot(df_ft_plv, df_ft_clust, df_ft_clust_stats, st_edges_result, 'PLV_Ind_', axs, 2)      
    
plt.tight_layout(rect=[0,0.09,1,1])

#fig.suptitle('PSD in brain regions by spectrum')
plt.savefig(f'features_in_stages{n_stages}.png') 


# Results

In [ ]:
# Calculate stage distances & clustering quality metrics

# Main data
df_adj_st_metr = calc_stage_metr_noground(df_features, st_edges_result)
st_metr_silh = df_adj_st_metr['Silh'].to_numpy()
st_metr_calh = df_adj_st_metr['Cal-Har'].to_numpy()
st_metr_davb = df_adj_st_metr['Dav-Bold'].to_numpy()

st_dist_ward, st_dist_centr = spr.calc_stage_distances(df_features, st_edges_result)
df_adj_st_metr['Centr'] = st_dist_centr
df_adj_st_metr['Ward'] = st_dist_ward

st_dist_centr_max = np.max(st_dist_centr)
st_dist_ward_max = np.max(st_dist_ward)
st_metr_silh_max = np.max(st_metr_silh)
st_metr_calh_max = np.max(st_metr_calh)
st_metr_davb_max = np.max(st_metr_davb)

display(df_adj_st_metr)
df_adj_st_metr.to_csv(f'df_adj_st_metr{n_stages}.csv')

## Main data

In [ ]:
# Plotting stages, distances, silh between adjacent

# df_features = df_ft_tot_pca
st_bands, st_labels = spr.form_stage_bands(st_edges_result, n_samples)
n_stages = len(st_bands)

# Sorting cluster labels for st_edges_all
#edg_labels_all = cluster_method.labels_
labels_uq = sorted(set(edg_labels_all), key=list(edg_labels_all).index)
edg_labels_sort = []
for _cl in range(len(labels_uq)):
    edg_labels_sort += [_cl for i in np.where(edg_labels_all == labels_uq[_cl])[0]]

# Getting time length of the stages
cluster_events = sec5_epochs.events.copy() # for m10 and m3
#cluster_events = epochs_filt_rr[:-2].events.copy() # for m8
cluster_events[:,2] = st_labels
time_bands = []
for i in range(n_stages):
    cl_samples = np.where(cluster_events[:,2] == i)[0]
    time_bands.append((cluster_events[cl_samples[0]][0], cluster_events[cl_samples[-1]][0], st_bands[i][2]))
st_time_len = [round((time_bands[i][1]-time_bands[i][0])/sampling_rate) for i in range(n_stages)]
print(st_time_len)

# Plotting stages
pal = plt.get_cmap('Set3')
#pal = plt.get_cmap('gist_ncar')

fig, ax = plt.subplots(figsize=(6,4))
for _st in range(n_stages):
    # Set x & y limits
    (ymin, ymax) = (-0.2, 1.1)
    ax.set_ylim(ymin, ymax)
    ax.set_xlim(0, n_samples)
    
    # Stages plot
    (x_start, x_end, st_name) = st_bands[_st]
    x = np.arange(x_start, x_end+1)
    y = np.full(len(x), 0)    
    event_label = 'N=%d' % (x_end-x_start+1)
    ax.add_patch(Rectangle((x_start, -0.05), x_end-x_start, 0.05, edgecolor = 'black', 
                           #facecolor = pal(_st/(n_stages-1)), fill=True, lw=1)) # for 9 stages
                           facecolor = pal(_st/(n_stages)), fill=True, lw=1)) # for 8 stages
    
    # Vertical lines (stage boundaries)
    ax.vlines(x_end, ymin=ymin, ymax=ymax, color='black', linewidth=1) # black lines
    #ax.vlines(x_end, ymin=ymin, ymax=ymax, color=pal(_st/(n_stages-1))) # colored lines
    
    # Background colors
    #ax.axvspan(x_start, x_end, alpha=0.3, color=pal(_st/(n_stages-1)))

    # Scatter plot
    x_sc = [st_edges_all[i] for i in range(len(st_edges_all)) if edg_labels_sort[i]==_st]
    y_sc = np.full(len(x_sc), 0.4)
    s = [1.5*x_sc.count(x_sc[i]) for i in range(len(x_sc))]
    #ax.scatter(x_sc, y_sc, s=s, color=pal(_st/(n_stages-1))) # for 9 stages
    ax.scatter(x_sc, y_sc, s=s, color=pal(_st/(n_stages))) # for 8 stages
    
    # Add text (st_name & st_length in sec)
    plt.text((x_start+x_end)/2, -0.12, st_name, fontsize=10, fontweight='bold', horizontalalignment='center')
    plt.text((x_start+x_end)/2, 0.02, str(st_time_len[_st])+'s', fontsize=9, fontstyle='italic', horizontalalignment='center')
    
# Adjacent stage distances (Centroid) & Silh coef 
x_dist = st_edges_result[1:-1]
y_centr = [_dist/st_dist_centr_max for _dist in st_dist_centr]
y_silh = [_dist/st_metr_silh_max for _dist in st_metr_silh]
#y_calh = [_dist/st_metr_calh_max for _dist in st_metr_calh]
y_davb = [_dist/st_metr_davb_max for _dist in st_metr_davb]
    
ax.plot(x_dist, y_centr, linestyle='--', marker='o', color='green', label='Centroid distance')
ax.plot(x_dist, y_silh, linestyle='--', marker='o', color='blue', label='Silhouette score')
#ax.plot(x_dist, y_calh, linestyle='--', marker='o', color='orange', label='Cal-Har metric')
ax.plot(x_dist, y_davb, linestyle='--', marker='o', color='red', label='Davies-Bouldin score')

ax.set(xlabel='Epoches')
ax.tick_params(axis='both', labelsize=11, direction='in')
plt.savefig('alg6.png')

#plt.tight_layout(rect=[-0.01,0.05,1,1])
#handles, labels = ax.get_legend_handles_labels()
#leg = fig.legend(handles, labels, loc='lower center', ncol=3, fontsize=11, handlelength=4)# , mode='expand')


In [ ]:
marked_epochs_yasa = pd.read_csv('yasa_for_plotting.csv').rename(columns={'Unnamed: 0':'epoch'})

In [ ]:

import plotly.graph_objects as go
fig = go.Figure()


for _st in range(n_stages):
    # Set x & y limits
    ymin, ymax = -0.1, 1.1
    fig.update_layout(yaxis=dict(range=[ymin, ymax]), xaxis=dict(range=[0, n_samples]))
    
    # Stages plot
    x_start, x_end, st_name = st_bands[_st]
    print(x_start, x_end, st_name)
    x = np.arange(x_start, x_end + 1)
    y = np.full(len(x), 0)
    event_label = 'N=%d' % (x_end - x_start + 1)
    
    fig.add_shape(type='rect',
                  x0=x_start, y0=-0.05, x1=x_end, y1=0,
                  line=dict(color='black', width=1),
                  fillcolor='red',
                  opacity=1,
                  layer='below'
                  )
    
    # Vertical lines (stage boundaries)
    fig.add_shape(type='line',
                  x0=x_start+5, y0=ymin + 0.05, x1=x_start+5, y1=ymax,
                  line=dict(color='red')
                  )
    
    # Background colors
    fig.add_shape(type='rect',
                  x0=x_start, y0=ymin, x1=x_end, y1=ymax,
                  fillcolor='white',
                  opacity=0.3,
                  layer='below'
                  )
    
    
    fig.add_annotation(text=str(st_time_len[_st]//60) + 'm', x=(x_start + x_end) / 2, y=-0.02,
                       font=dict(size=12, color='black', family='Balto'),
                       showarrow=False, align='center')

colors = {'N1':'lightblue',
          'N2':'blue',
          'N3': 'darkblue',
          'W': 'orange',
          'R': 'purple'}


for t in marked_epochs_yasa['best'].unique():
    dfp = marked_epochs_yasa[marked_epochs_yasa['best']==t]
    fig.add_traces(go.Bar(x=dfp['epoch']*6, y = dfp['prob'], name=t,
                         marker_color=colors[t]))

# Set the figure size
fig.update_layout(width=1100, height=600)
fig.update_layout(xaxis=dict(title='Stages'), yaxis=dict(title='yasa predictions'), showlegend=False)
# Show the figure
fig.show()
fig.write_image('compare_6_9_stages.png')

In [ ]:
# Getting time length of the stages

st_names =['Stage '+str(i+1) for i in range(n_stages)]
df_st_time_len = pd.DataFrame(columns=['Parameters']+st_names)

time_bands = []
for i in range(n_stages):
    cl_samples = np.where(cluster_events[:,2] == i)[0]
    time_bands.append((cluster_events[cl_samples[0]][0], cluster_events[cl_samples[-1]][0], 'St'+str(i+1)))

# Start-end time
st_param_dict = dict([(st_names[i], (round(time_bands[i][0]/sampling_rate), round(time_bands[i][1]/sampling_rate))) 
                      for i in range(n_stages)])
new_row = {'Parameters': 'Start-end time, sec'}
new_row.update(st_param_dict)
df_st_time_len = pd.concat([df_st_time_len, pd.DataFrame(new_row)], ignore_index = True)

# Time length, sec
st_param_dict = dict([(st_names[i], str((time_bands[i][1]-time_bands[i][0])/sampling_rate)) for i in range(n_stages)])
new_row = {'Parameters': 'Time length, sec'}
new_row.update(st_param_dict)
df_st_time_len = pd.concat([df_st_time_len, pd.DataFrame(new_row, index=[0])], ignore_index = True)

# Number of epochs, sec
st_param_dict = dict([(st_names[i], st_bands[i][1]-st_bands[i][0]+1) for i in range(n_stages)])
new_row = {'Parameters': 'Number of epochs'}
new_row.update(st_param_dict)
df_st_time_len = pd.concat([df_st_time_len, pd.DataFrame(new_row, index=[0])], ignore_index = True)

display(df_st_time_len)    
df_st_time_len.to_csv(f'df_st_time_len{n_stages}.csv')


In [ ]:
# DataFrame with stage distances (by adjacent pairs)

# Forming DataFrame with distance values    
st_dist_names = ['St'+str(i)+'_St'+str(i+1) for i in range(1, n_stages)]
df_st_dist_pairs = pd.DataFrame(columns=['Method'] + st_dist_names)
#display(df_st_dist_pairs)
 
# Ward distances
st_dist_ward_dict = dict([(st_dist_names[i], st_dist_ward[i]) for i in range(len(st_dist_names))])
new_row = {'Method': 'Ward distance'}
new_row.update(st_dist_ward_dict)
df_st_dist_pairs = pd.concat([df_st_dist_pairs, pd.DataFrame(new_row, index=[0])], ignore_index = True)

# Centroid distances
st_dist_cen_dict = dict([(st_dist_names[i], st_dist_centr[i]) for i in range(len(st_dist_names))])
new_row = {'Method': 'Centroid distance'}
new_row.update(st_dist_cen_dict)
df_st_dist_pairs = pd.concat([df_st_dist_pairs, pd.DataFrame(new_row, index=[0])], ignore_index = True)

# Silh metrics
st_metr_silh_dict = dict([(st_dist_names[i], st_metr_silh[i]) for i in range(len(st_dist_names))])
new_row = {'Method': 'Silhouette Coefficient'}
new_row.update(st_metr_silh_dict)
df_st_dist_pairs = pd.concat([df_st_dist_pairs, pd.DataFrame(new_row, index=[0])], ignore_index = True)

# Cal-Har metrics
st_metr_calh_dict = dict([(st_dist_names[i], st_metr_calh[i]) for i in range(len(st_dist_names))])
new_row = {'Method': 'Calinski-Harabasz Index'}
new_row.update(st_metr_calh_dict)
df_st_dist_pairs = pd.concat([df_st_dist_pairs, pd.DataFrame(new_row, index=[0])], ignore_index = True)

# Dav-Bold metrics
st_metr_davb_dict = dict([(st_dist_names[i], st_metr_davb[i]) for i in range(len(st_dist_names))])
new_row = {'Method': 'Davies-Bouldin Index'}
new_row.update(st_metr_davb_dict)
df_st_dist_pairs = pd.concat([df_st_dist_pairs, pd.DataFrame(new_row, index=[0])], ignore_index = True)

display(df_st_dist_pairs)
df_st_dist_pairs.to_csv(f'df_st_dist_pairs{n_stages}.csv')

In [ ]:
# DataFrame with averaged cluster distances & metrics

col_names = ['Subject', 'Center type', 'Ward dist', 'Centr dist', 'Silh Stage', 'Cal-Har Stage', 'Dav-Bold Stage', 
             'Silh Total', 'Cal-Har Total', 'Dav-Bold Total']
df_st_dist_total = pd.DataFrame(columns=col_names)

new_row = {'Subject': 'pigarev', 'Center type': cl_center_type}
n_pair_metr = len(df_st_dist_pairs)
n_head_col = len(new_row)
n_add_col = len(col_names)-n_head_col-n_pair_metr 

st_metr_val = [df_st_dist_pairs[df_st_dist_pairs.columns[1:]].iloc[r].mean() for r in range(n_pair_metr)]
new_row_add1 = dict([(col_names[r+n_head_col], st_metr_val[r]) for r in range(n_pair_metr)])
new_row.update(new_row_add1)

df = cluster_metrics_noground('pigarev', df_features, st_labels)
new_row_add2 = dict([(col_names[r+n_head_col+n_pair_metr], df[df.columns[r+1]].iloc[0]) for r in range(n_add_col)])
new_row.update(new_row_add2)

df_st_dist_total = pd.concat([df_st_dist_total, pd.DataFrame(new_row, index=[0])], ignore_index = True)

display(df_st_dist_total)
df_st_dist_total.to_csv(f'df_st_dist_total{n_stages}.csv')


# Save results

In [ ]:

print(st_edges_result)
st_edges_res_samp = sec5_epochs[st_edges_result[:-1]].events[:,0]
st_edges_res_samp

In [ ]:

# Main data
np.savetxt(os.path.join(ft_dir_path, f'st_edges_result{n_stages}.txt'), st_edges_result)
np.savetxt(os.path.join(ft_dir_path, f'st_edges_res_samp{n_stages}.txt'), st_edges_res_samp)

